In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim import corpora
from gensim import models
import MeCab
from gensim.models import word2vec

/Users/mmiyazaki/dataiku/Design/DATA_DIR/code-envs/python/japan-nlp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [4]:
# Read recipe inputs
your_trip_advisor = dataiku.Dataset("trip_advisor_clustered_prepared")
df = your_trip_advisor.get_dataframe()

In [19]:
tfidf_path = dataiku.Folder("tMMk2S0T").get_path() + "/tf_idf"

In [8]:
list_vocabs = df[df['cluster_labels'].isin(['接客などのサービス', '具材・素材・味'])]['words_concat'].values

In [11]:
ramen_word = list_vocabs[0].split(",") + list_vocabs[1].split(",")

In [17]:
ramen_dictionary_path = dataiku.Folder("POe5uF4H").get_path() + "/ramen_dictionary"
dictionary = corpora.Dictionary.load(ramen_dictionary_path)
corpus = list(map(dictionary.doc2bow, [ramen_word]))

In [18]:
corpus

[[(2, 1),
  (3, 1),
  (6, 1),
  (12, 2),
  (23, 1),
  (25, 1),
  (34, 3),
  (36, 1),
  (41, 2),
  (52, 2),
  (56, 2),
  (62, 1),
  (63, 5),
  (64, 1),
  (79, 2),
  (87, 1),
  (90, 11),
  (126, 2),
  (132, 6),
  (135, 1),
  (139, 1),
  (170, 15),
  (187, 1),
  (194, 1),
  (206, 2),
  (213, 3),
  (220, 1),
  (222, 1),
  (228, 1),
  (246, 1),
  (256, 1),
  (257, 1),
  (283, 1),
  (290, 1),
  (291, 3),
  (320, 1),
  (325, 6),
  (327, 1),
  (330, 1),
  (332, 5),
  (337, 1),
  (356, 1),
  (389, 1),
  (392, 2),
  (412, 1),
  (413, 4),
  (420, 1),
  (424, 6),
  (425, 1),
  (448, 1),
  (459, 3),
  (462, 3),
  (464, 4),
  (469, 1),
  (489, 1),
  (504, 2),
  (540, 2),
  (623, 1),
  (627, 1),
  (655, 1),
  (680, 1),
  (681, 1),
  (719, 8),
  (758, 1),
  (782, 3),
  (819, 2),
  (842, 1),
  (853, 1),
  (881, 2),
  (1008, 1),
  (1015, 1),
  (1025, 1),
  (1095, 1),
  (1117, 1),
  (1189, 1),
  (1312, 1),
  (1372, 1),
  (1394, 1),
  (1406, 1),
  (1426, 2),
  (1471, 1),
  (1487, 1),
  (1512, 7),
  (1561,

In [20]:
models.load(tfidf_path)

AttributeError: module 'gensim.models' has no attribute 'load'

In [0]:
wakati_folder = dataiku.Folder("0kM5kXKs").get_path()
tagger_path = '-Owakati -d ' + wakati_folder

In [0]:
tagger = MeCab.Tagger(tagger_path)#タグはMeCab.Tagger（neologd辞書）を使用
tagger.parse('')

In [0]:
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [0]:
wakati_ramen_text = []
for i in df['jp']:
    txt = tokenize(i, 2, 10000, True)
    wakati_ramen_text.append(txt)

In [0]:
#[w for w in sublist in wakati_ramen_text]
vocab = [w for sublist in wakati_ramen_text for w in sublist]

In [0]:
words = []
vectors = []
for word in vocab:
    try:
        vector = ramen_model.wv[word]
        words.append(word)
        vectors.append(vector)
    except KeyError:
        None

In [0]:
vocab_df = pd.DataFrame(vectors)
vocab_df['words'] = words

In [0]:
trip_advisor_tf_idf = dataiku.Dataset("trip_advisor_vocabs")
trip_advisor_tf_idf.write_with_schema(vocab_df)

In [0]:
#ramen_dictionary_path = dataiku.Folder("POe5uF4H").get_path() + "/ramen_dictionary"
#dictionary = corpora.Dictionary.load(ramen_dictionary_path)
#corpus = list(map(dictionary.doc2bow, reviews_concat))